In [1]:
using JuMP, Gurobi

# First we define main parameters

In [2]:
#Constant of the problem
n_region = 6
T = 6 #We consider a 6 hour problem
VE = 10
R = [VE for i in 1:T] #Maximum vehicles available (randomly chose atm)
WT = 2 #weight for incident state

#Parameters that we fix for the algorithm
DELTA = 60 #Units of time period
sim_periods = 60 #Time periods of discretization in the SIR simulation
sim_time = DELTA/sim_periods #Used so that if we discretize at very granular time, the cost doesn't grow too much
DIS = 0.01 #Discreztization gap for creating the states-space
espilon = 0.0000001 #Error term
weight = [1.00 for i in  1:n_region] #Weight of the region i
DP_gap = 0.00001
negative_column = [false]
range_it = [[[0, VE] for t in 1:T] for n in 1:n_region]
Tot_plans = 500000
record_now = [1]
record_end = [1]
NCOLB = 5000
lb_node = [0.0 for n in 1:NCOLB]
pre_node = [1 for n in 1:NCOLB]
sign_node = [true for n in 1:NCOLB]
thre_node = [[1, 1, 1.0] for n in 1:NCOLB]
error_fra = 0.00000001
cur_node = [1]
tot_node = [0]
UB = [Inf]
LB = [-Inf]
already_optimized = [false]
null_constraints = []
vehicles_constraints = []
region_constraints = []
new_cg = [false];

In [3]:
plan_set = [[[0 for t in 1:T] for i in 1:n_region], 
            [[1 for t in 1:T] for i in 1:n_region], 
            [[2 for t in 1:T] for i in 1:n_region]]

n_plan = length(plan_set)
;

In [4]:
#SIR parameters
alpha = [0.001]
betaK = [0.17]
Lambda = [0.0001]
mu = [0.1]
for i in 2:n_region
    push!(alpha, alpha[i-1]*0.95)
    push!(betaK, betaK[i-1]*0.95)
    push!(Lambda, Lambda[i-1]*0.95)
    push!(mu, mu[i-1]*0.95)
end

In [5]:
cost_p = [1.0 for p in 1:Tot_plans]
vehn_pt = [[1 for t in 1:T] for p in 1:Tot_plans]
regn_p = [0 for t in 1:Tot_plans]

infea_p = [true for p in 1:Tot_plans];

In [6]:
#Initial State for each region

initial_state = [[0.99, 0.01, 0, 0]]
for r in 2:n_region
    f0_r = round(last(initial_state)[1] - 0.06, digits = 2)
    c0_r = round(last(initial_state)[2] + 0.03, digits = 2)
    r0_r = round(last(initial_state)[3] + 0.02, digits = 2)
    h0_r = round(last(initial_state)[4] + 0.01, digits = 2)
    push!(initial_state, [f0_r, c0_r, r0_r, h0_r])
end

# Simulation Module

In [7]:
function rounding_procedure(L, DIS)
    a = L[1]
    b = L[2]
    c = L[3]
    d = L[4]
    rou_a = a/DIS
    rou_b = b/DIS
    rou_c = c/DIS
    rou_d = d/DIS
    res = [0.0 for i in 1:4]
    res[1], res[2], res[3], res[4] = rou_a%1, rou_b%1, rou_c%1, rou_d%1
    residual = round(res[1] + res[2] + res[3] + res[4], digits = 0)
    rou_a = floor(rou_a)
    rou_b = floor(rou_b)
    rou_c = floor(rou_c)
    rou_d = floor(rou_d)
    while residual > 0
        max_ind = -1
        max_val = -1
        for i in 1:4
            if res[i] > max_val
                max_val = res[i]
                max_ind = i
            end
        end
        
        if max_ind == 1
            rou_a = rou_a + 1
        elseif max_ind == 2
            rou_b = rou_b + 1
        elseif max_ind == 3
            rou_c = rou_c + 1
        elseif max_ind == 4
            rou_d = rou_d + 1
        end
        
        res[max_ind] = - 1
        residual = residual - 1
    end
    a, b, c, d = round(rou_a*DIS, digits = 2), round(rou_b*DIS, digits = 2), round(rou_c*DIS, digits = 2), round(rou_d*DIS, digits = 2)
    return (a,b,c,d)
end

rounding_procedure (generic function with 1 method)

In [8]:
function sir(state, x, region)
    wcost = 0
    alpha_r, betaK_r, lambda_r, mu_r = alpha[region], betaK[region], Lambda[region], mu[region]
    f_t, c_t, r_t, h_t = state[1], state[2], state[3], state[4]
    for t in 1:sim_periods
        if (h_t + (alpha_r*f_t - lambda_r*x)*sim_time) >= 0
            f_t_1 = f_t + (-alpha_r*f_t - betaK_r*f_t*(c_t + h_t))*sim_time
            c_t_1 = c_t + (-mu_r*c_t + betaK_r*f_t*(c_t + h_t) + lambda_r*x)*sim_time
            r_t_1 = r_t + (mu_r*c_t)*sim_time
            h_t_1 = h_t + (alpha_r*f_t - lambda_r*x)*sim_time
        else
            cut_zero = alpha_r*f_t + h_t/sim_time
            f_t_1 = f_t + (-alpha_r*f_t - betaK_r*f_t*(c_t + h_t))*sim_time
            c_t_1 = c_t + (-mu_r*c_t + betaK_r*f_t*(c_t + h_t)) + cut_zero)*sim_time
            r_t_1 = r_t + (mu_r*c_t)*sim_time
            h_t_1 = 0
        end
        wcost += (WT*h_t + c_t)*sim_time
        f_t, c_t, r_t, h_t = f_t_1, c_t_1, r_t_1, h_t_1
    end
    L = [f_t, c_t, r_t, h_t]
    f_t, c_t, r_t, h_t = rounding_procedure(L, DIS)
    return (f_t, c_t, r_t, h_t, wcost)
end

sir (generic function with 1 method)

# Generate the space of all possible states at each Time Period

In [9]:
function state_space(initial_state)
    #Generate lower bounds for f, h and upper bounds for r
    LB_f = []
    UB_r = []
    LB_h = []
    for i in 1:n_region
        f_min_i = [initial_state[i][1]]
        r_max_i = [initial_state[i][3]]
        h_min_i = [initial_state[i][4]]
        tem_f , tem_c, tem_r, tem_h = initial_state[i][1], initial_state[i][2], initial_state[i][3], initial_state[i][4]
        for time in 1:T
            tem_f, tem_c, tem_r, tem_h, cost = sir([tem_f, tem_c, tem_r, tem_h], VE, i)
            push!(f_min_i, tem_f)
            push!(r_max_i, tem_r)
            push!(h_min_i, tem_h)
        end
        push!(LB_f, f_min_i)
        push!(UB_r, r_max_i)
        push!(LB_h, h_min_i)
    end
    
    #Generate upper bounds for f, h and lower bounds for r
    UB_f = []
    LB_r = []
    UB_h = []    
    for i in 1:n_region
        f_max_i = [initial_state[i][1]]
        r_min_i = [initial_state[i][3]]
        h_max_i = [initial_state[i][4]]
        tem_f , tem_c, tem_r, tem_h = initial_state[i][1], initial_state[i][2], initial_state[i][3], initial_state[i][4]
        for time in 1:T
            tem_f, tem_c, tem_r, tem_h, cost = sir([tem_f, tem_c, tem_r, tem_h], 0, i)
            push!(f_max_i, tem_f)
            push!(r_min_i, tem_r)
            push!(h_max_i, tem_h)
        end
        push!(UB_f, f_max_i)
        push!(LB_r, r_min_i)
        push!(UB_h, h_max_i)
    end
    
    #Now that we have bounds for f, h and r, we can create the whole discretization space
    values_possible = round(1/DIS)
    state_space = []
    number_of_states = 0
    for i in 1:n_region
        state_i = []
        for time in 1:(T+1)
            state_i_time = []
            for v1 in 0:values_possible
                for v2 in 0:(values_possible - v1)
                    for v3 in 0:(values_possible - v1 - v2)
                        v4 = values_possible - v1 - v2 - v3
                        f_el = round(v1*DIS, digits = 2)
                        c_el = round(v2*DIS, digits = 2)
                        r_el = round(v3*DIS, digits = 2)
                        h_el = round(v4*DIS, digits = 2)
                        if (f_el <= UB_f[i][time]) & (f_el >= LB_f[i][time]) & (r_el <= UB_r[i][time]) & (r_el >= LB_r[i][time]) & (h_el <= UB_h[i][time]) & (h_el >= LB_h[i][time])
                            push!(state_i_time, [f_el, c_el, r_el, h_el])
                            number_of_states+=1
                        end
                    end
                end
            end
            push!(state_i, state_i_time)
        end
        push!(state_space, state_i)
    end
    return state_space, number_of_states
end

state_space (generic function with 1 method)

In [10]:
state_space_fcrh, states_number = state_space(initial_state);

# Compute all possible transitions and the associated cost

In [43]:
function DP_All_sim(state_space)
    state_store_next = [[[[[] for x in 0:VE] for s in 1:length(state_space[i][t])] for t in 1:T] for i in 1:n_region]
    state_store_cost = [[[[0.0 for x in 0:VE] for s in 1:length(state_space[i][t])] for t in 1:T] for i in 1:n_region]
    for i in 1:n_region
        count_sim = 0
        Threads.@threads for t in T:-1:1
            for s in 1:length(state_space[i][t])
                x_max = VE
                for x in 0:x_max
                    count_sim += 1
                    f, c, r, h, cost = sir(state_space[i][t][s], x, i)
                    cost = cost*weight[i]
                    next = findall(x-> x==[f, c, r, h], state_space[i][t+1])
                    state_store_next[i][t][s][x+1] = next
                    state_store_cost[i][t][s][x+1] = cost
                end
            end
        end
        println("Region ", i, " runs ", count_sim, " simulations")
    end
    return state_store_next, state_store_cost
end

DP_All_sim (generic function with 1 method)

In [44]:
state_store_ss1, state_store_cost = DP_All_sim(state_space_fcrh);

Region 1 runs 58124 simulations
Region 2 runs 27599 simulations
Region 3 runs 27148 simulations
Region 4 runs 24398 simulations
Region 5 runs 25212 simulations
Region 6 runs 20042 simulations


# CG Starts Here

# Add initial columns

In [13]:
function dummy_columns()
    for i in 1:n_region
    regn_p[record_end[1]] = i
        for t in 1:T
            vehn_pt[record_end[1]][t] = 0
        end
        cost_p[record_end[1]] = 999999
        infea_p[record_end[1]] = false
        record_end[1] += 1
    end
end

dummy_columns (generic function with 1 method)

In [14]:
function ini_columns(initial_state)
    ini_tot_cost = 0
    for i in 1:n_region
        regn_p[record_end[1]] = i
        tem_f, tem_c, tem_r, tem_h = initial_state[i][1], initial_state[i][2], initial_state[i][3], initial_state[i][4]
        tem_cost = 0
        
        for t in 1:T
            vehn_pt[record_end[1]][t] = 1
            tem_f_1, tem_c_1, tem_r_1, tem_h_1, tem_cost_1 = sir([tem_f, tem_c, tem_r, tem_h], vehn_pt[record_end[1]][t], i)
            tem_f, tem_c, tem_r, tem_h = tem_f_1, tem_c_1, tem_r_1, tem_h_1
            tem_cost += tem_cost_1
        end
        cost_p[record_end[1]] = tem_cost
        infea_p[record_end[1]] = false
        
        ini_tot_cost += cost_p[record_end[1]]*weight[i]
        record_end[1] += 1
    end
    UB[1] = ini_tot_cost
end     

ini_columns (generic function with 1 method)

# Define the Master Problem

In [15]:
function RMP_define_model()
    RMP = Model(Gurobi.Optimizer)
    set_optimizer_attribute(RMP, "OutputFlag", 0)
    z_p = @variable(RMP, 0 <= z[1:(record_end[1]-1)] <= 1)
    return RMP, z_p
end

RMP_define_model (generic function with 1 method)

In [16]:
function RMP_add_columns(model, z, c_time, c_region)
    println("RMP begins: Currently, the total number of plans is ", record_end[1] - 1, " from ", record_now[1] - 1)
    #We update the constraints of our model
    if already_optimized[1]
        delete(model, c_time)
        unregister(model, :c_time)
        delete(model, c_region)
        unregister(model, :c_region)
        for p in record_now[1]:(record_end[1]-1)
            push!(z, @variable(model, base_name = "z[$(p)]", lower_bound = 0, upper_bound = 1))
        end
    end
        
    #Select the region for each plan
    for p in record_now[1]:(record_end[1]-1)
        if infea_p[p]
            @constraint(model, z[p] <= 0)
            continue
        end
    end
    
    #VE vehicles can be used at time t
    @constraint(model, c_time[i=1:T], sum(z[p]*vehn_pt[p][i] for p in 1:(record_end[1]-1)) <= R[i])
    
    #One plan per region
    @constraint(model, c_region[i=1:n_region], sum(z[p] for p in 1:(record_end[1]-1) if regn_p[p] == i) == 1)
    
    
    #We update the objective of our model
    @objective(model, Min, sum(weight[regn_p[p]]*z[p]*cost_p[p] for p in 1:(record_end[1]-1)))
    record_now[1] = record_end[1]
    
    optimize!(model)
    
    if termination_status(model) == MOI.OPTIMAL
        Total_cost = objective_value(model)
        println("The optimal objective is ", Total_cost)
        
        pi_ = [dual(c_time[i]) for i in 1:T]
        phi = [dual(c_region[i]) for i in 1:n_region]
        
    else
        println("No Feasible Solution for Gurobi!!!")
    end
    
    negative_column[1] = false
    already_optimized[1] = true
    
    return model, z, pi_, phi, c_time, c_region
end

RMP_add_columns (generic function with 1 method)

# Pricing Problem to generate the new column to add

In [17]:
function Backward_DP(region, initial_state, pi, phi, state_space, state_store_next, state_store_cost)
    #We initilize the costs
    state_cost = [[Inf for s in 1:length(state_space[region][t])] for t in 1:(T+1)]
    state_backward = [[0.0 for s in 1:length(state_space[region][t])] for t in 1:T]
    state_decision = [[0.0 for s in 1:length(state_space[region][t])] for t in 1:T]
    
    for s in 1:length(state_space[region][T+1])
        state_cost[T+1][s] = -phi[region]
    end
    
    for t in T:-1:1
        for s in 1:length(state_space[region][t])
            x_max = VE
            min_cost = Inf
            min_state = -1
            min_x = -1
            for x in 0:VE
                if (x < range_it[region][t][1]) | (x > range_it[region][t][2]) #Check Branching Constraints
                    continue
                end
                tem_cost = state_store_cost[region][t][s][x+1]
                tem_next = state_store_next[region][t][s][x+1]
                if length(tem_next) == 0
                    continue
                end
                tem_cost = tem_cost + state_cost[t+1][tem_next[1]] - pi[t]*x
                if tem_cost < min_cost
                    min_cost = tem_cost
                    min_state = tem_next[1]
                    min_x = x
                end
            end
            state_cost[t][s] = min_cost
            state_backward[t][s] = min_state
            state_decision[t][s] = min_x
        end
    end
    
    #We now compute the best decision starting from the initial state
    state_opt = 1
    #println("BackwardDP Opt cost is ", state_cost[1][state_opt], " for region ", region)
    temp_cost = 0
    decision = [0.0 for t in 1:T]
    if state_cost[1][state_opt] < -0.00001
        for t in 1:T
            decision[t] = state_decision[t][state_opt]
            state_opt = state_backward[t][state_opt]
            state_opt = Int(state_opt)
        end  
        
        f, c, r, h = initial_state[region][1], initial_state[region][2], initial_state[region][3], initial_state[region][4]
        for t in 1:T
            f_t, c_t, r_t, h_t, cost_t = sir([f, c, r, h], decision[t], region)
            f, c, r, h = f_t, c_t, r_t, h_t
            temp_cost += cost_t
        end
    end
    
    return decision, temp_cost
end     

Backward_DP (generic function with 1 method)

In [47]:
function BackwardAll(initial_state, pi, phi, state_space, store_next_state, store_state_cost)
    new_plan = []
    cost = []
    for i in 1:n_region #Can do it in Parallel
        plan_i, cost_i = Backward_DP(i, initial_state, pi, phi, state_space, store_next_state, store_state_cost)
        push!(new_plan, plan_i)
        push!(cost, cost_i)
    end
    return new_plan, cost
end

BackwardAll (generic function with 1 method)

In [38]:
#This gives you the new plan to add to your pool for every region
function extract_column(new_plan, costs)
    for i in 1:n_region
        if costs[i] == 0
            continue
        end
        negative_column[1] = true
        regn_p[record_end[1]] = i
        for t in 1:T
            vehn_pt[record_end[1]][t] = new_plan[i][t]
        end
        cost_p[record_end[1]] = costs[i]
        infea_p[record_end[1]] = false
        record_end[1] += 1
    end
end

extract_column (generic function with 1 method)

In [39]:
function input()
    record_now[:] = [1]
    record_end[:] = [1]
    negative_column[:] = [false]
    range_it[:] = [[[0, VE] for t in 1:T] for n in 1:n_region]
    lb_node[:] = [0.0 for n in 1:NCOLB]
    pre_node[:] = [1 for n in 1:NCOLB]
    sign_node[:] = [true for n in 1:NCOLB]
    thre_node[:] = [[1, 1, 1.0] for n in 1:NCOLB]
    cur_node[:] = [1]
    tot_node[:] = [0]
    UB[:] = [Inf]
    LB[:] = [-Inf]
    already_optimized[:] = [false]
    c_time[:] = []
    c_region[:] = []
    cost_p[:] = [1.0 for p in 1:Tot_plans]
    vehn_pt[:] = [[1 for t in 1:T] for p in 1:Tot_plans]
    regn_p[:] = [0 for t in 1:Tot_plans]
    infea_p[:] = [true for p in 1:Tot_plans]
    new_cg = [false]
end

input (generic function with 1 method)

# Column Generation

In [40]:
function column_generation(initial_state, state_space, store_next_state, store_state_cost, c_time, c_region)
    model, z = RMP_define_model()
    while 1==1
        model, z, pi_, phi, c_time, c_region = RMP_add_columns(model, z, c_time, c_region)
        new_plan, costs = BackwardAll(initial_state, pi_, phi, state_space, store_next_state, store_state_cost)
        extract_column(new_plan, costs)
        println("-----------------------------------------------------")
        if !negative_column[1]
            break
        end
    end
    println("Column Generations Ends")
    println("-----------------------------------------------------")
    return model, z, c_time, c_region
end

column_generation (generic function with 2 methods)

In [42]:
c_time = []
c_region = []
input()
dummy_columns()
ini_columns(initial_state)
#model, z = RMP_define_model()
time = @elapsed column_generation(initial_state, state_space_fcrh, state_store_ss1, state_store_cost, c_time, c_region);
println(time)

Academic license - for non-commercial use only
RMP begins: Currently, the total number of plans is 12 from 0
The optimal objective is 342.5778020849276
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 18 from 12
The optimal objective is 320.0697945387827
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 24 from 18
The optimal objective is 306.1668954588298
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 30 from 24
The optimal objective is 300.9494766313133
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 36 from 30
The optimal objective is 295.78401583603073
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 42 from 36
The optimal objective is 289.964189207376
------------------------------------------------

# Branch & Price Part

# Algorithm to find the most fractional solution

In [23]:
function check_fractional_sol(model, z_pi)
    fra_region = -1
    fra_time = -1
    fra_lbval = -1
    
    if termination_status(model) == MOI.OPTIMAL
        LB[1] = objective_value(model)
        assign_it = [[0.0 for t in 1:T] for i in 1:n_region]
        
        #Look at the value the CG assigned to each plan in the end
        for p in 1:record_end[1]
            if infea_p[p]
                continue
            end
            current = value.(z_pi[p])
            if current > 0.00001
                for t in 1:T
                    assign_it[regn_p[p]][t] = assign_it[regn_p[p]][t] + vehn_pt[p][t]*current
                end
            end
        end
        
        #Select the most fractional value, the region and the time period associated with it
        close_gap = 0.5
        middle_point = 0.5
        for i in 1:n_region
            for t in 1:T
                if ((assign_it[i][t]%1) > error_fra) & ((assign_it[i][t]%1) < (1 - error_fra))
                    if abs((assign_it[i][t]%1) - middle_point) < close_gap
                        close_gap = abs((assign_it[i][t]%1) - middle_point)
                        fra_region = i
                        fra_time = t
                        fra_lbval = floor(assign_it[i][t])
                    end
                end
            end
        end
    else
        println("No feasible solution for GUROBI")
    end
    #println("Branching: Region ", fra_region, " period ", fra_time, " floor ", fra_lbval)
    return (Int(round(fra_region)), Int(round(fra_time)), fra_lbval, assign_it)
end

check_fractional_sol (generic function with 1 method)

# In case the solution is not integer, we must add a branching constraint

In [24]:
function left_branching()
    fra_region = Int(round(thre_node[cur_node[1]][1]))
    fra_time = Int(round(thre_node[cur_node[1]][2]))
    fra_lbval = thre_node[cur_node[1]][3]
    
    #We add one left branching constraint
    range_it[fra_region][fra_time][2] = min(fra_lbval, range_it[fra_region][fra_time][2])
    
    if range_it[fra_region][fra_time][2] < range_it[fra_region][fra_time][1]
        println("Warnings: Infeasible branching constraints. Left")
    end
    
    #We prune the columns
    for p in 1:(record_end[1]-1)
        #Do not do it for the dummy plans
        if p < n_region + 1
            continue
        end
        
        #Only for the region that is concerned by the new constraint
        if regn_p[p] != fra_region
            continue
        end
        if vehn_pt[p][fra_time] > range_it[fra_region][fra_time][2]
            infea_p[p] = true
        end
    end
end

left_branching (generic function with 1 method)

# We update the LB and add new constraints

In [25]:
function best_LB_branching()
    
    range_it[:] = [[[0, VE] for t in 1:T] for n in 1:n_region]
    back_node = cur_node[1]
    
    #We need to add a list of new constraints on the range of vehicles available
    while back_node != 1
        fra_region = Int(round(thre_node[back_node][1]))
        fra_time = Int(round(thre_node[back_node][2]))
        fra_lbval = thre_node[back_node][3]
        fra_sign = sign_node[back_node]
        if fra_sign
            range_it[fra_region][fra_time][2] = min(fra_lbval, range_it[fra_region][fra_time][2]) #We add the left branching constraint
        else
            range_it[fra_region][fra_time][1] = max(fra_lbval, range_it[fra_region][fra_time][1]) #We add the right branching constraint
        end
        
        if range_it[fra_region][fra_time][2] < range_it[fra_region][fra_time][1] # We check that our two branching constraints are feasible
            println("Warnings: Infeasible Branching constraints. BestLB")
        end
        back_node = pre_node[back_node]
    end
    
    #Now we prune the columns
    for p in 1:(record_end[1]-1)
        infea_p[p] = false
        #Done to avoid doing it for the dummy columns at the beginning
        if p < n_region + 1
            continue
        end
        
        #Check wether or not the old plans are still feasible with the new constraints
        i_R = regn_p[p]
        for t in 1:T
            if (vehn_pt[p][t] < range_it[i_R][t][1]) | (vehn_pt[p][t] > range_it[i_R][t][2])
                infea_p[p] = true
                break
            end
        end
    end
end

best_LB_branching (generic function with 1 method)

# Branch & Price Algorithm

In [35]:
function branch_and_price(initial_state, state_space, next_states, state_costs)
    BB_nodes = [0]
    best_assign_it = [[0 for t in 1:T] for i in 1:n_region]
    
    c_time = []
    c_region = []
    input()
    dummy_columns()
    ini_columns(initial_state)
    model, z = RMP_define_model()
    
    while 1==1
        popfirst!(BB_nodes)
        
        c_time = []
        c_region = []
        model, z, c_time, c_region = column_generation(initial_state, state_space, next_states, state_costs, c_time, c_region)
        
        fra_region, fra_time, fra_lbval, assign_it = check_fractional_sol(model, z)
        
        if (LB[1] < UB[1]) & (fra_region >= 0) #Solution is still fractional
            println("Branching region: ", fra_region, " in time ", fra_time, " with value ", fra_lbval)
            
            #We add left node first
            tot_node[1] += 1
            lb_node[tot_node[1]] = LB[1]
            pre_node[tot_node[1]] = cur_node[1]
            sign_node[tot_node[1]] = true
            thre_node[tot_node[1]][1] = fra_region
            thre_node[tot_node[1]][2] = fra_time
            thre_node[tot_node[1]][3] = fra_lbval
            insert!(BB_nodes, 1, tot_node[1]) #Depth-first strategy
            
            #We then add the right node
            tot_node[1] += 1
            lb_node[tot_node[1]] = LB[1]
            pre_node[tot_node[1]] = cur_node[1]
            sign_node[tot_node[1]] = false
            thre_node[tot_node[1]][1] = fra_region
            thre_node[tot_node[1]][2] = fra_time
            thre_node[tot_node[1]][3] = fra_lbval + 1
            
            #We rank the nodes according to their lower bound
            for s in 1:length(BB_nodes)
                if s == length(BB_nodes)
                    push!(BB_nodes, tot_node[1])
                    break
                elseif lb_node[tot_node[1]] >= lb_node[BB_nodes[s]]
                    continue
                else
                    insert!(BB_nodes, s, tot_node[1])
                    break
                end
            end
        end
            
        if (LB[1] < UB[1]) & (fra_region < 0) #An integer solution is found
             UB[1] = LB[1] #We update the upper bounds and the decision
            for i in 1:n_region
                for t in 1:T
                    best_assign_it[i][t] = round(assign_it[i][t])
                end
            end
            println("Update UB ", UB[1])
        end
        
        println("Nodes in the queue ", length(BB_nodes), " with UB ", UB[1])
        
        if length(BB_nodes) > 0 #While we still have some nodes on the tree
            if pre_node[BB_nodes[1]] == cur_node[1] #Means we are going for the left branching
                cur_node[1] = BB_nodes[1]
                left_branching()
                println("Proceed left Branching!")
            else #Means that we are going for the lower bound branching
                cur_node[1] = BB_nodes[1]
                best_LB_branching()
                println("Proceed Best LB branching!")
            end
            
        else
            break
        end
        record_now[1] = 1
        already_optimized[1] = false
        #new_cg[1] = true
    end
    
    println("The Optimal solution is ", UB[1])
    
    for i in 1:n_region
        println("Region ", i, ":")
        for t in 1:T
            println(" ", best_assign_it[i][t])
        end
    end
    return best_assign_it
end

branch_and_price (generic function with 1 method)

In [48]:
c_time = []
c_region = []
@time final_decision = branch_and_price(initial_state, state_space_fcrh, state_store_ss1, state_store_cost);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
RMP begins: Currently, the total number of plans is 12 from 0
The optimal objective is 342.5778020849276
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 18 from 12
The optimal objective is 320.0697945387827
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 24 from 18
The optimal objective is 306.1668954588298
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 30 from 24
The optimal objective is 300.9494766313133
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 36 from 30
The optimal objective is 295.78401583603073
-----------------------------------------------------
RMP begins: Currently, the total number of plans is 42 from 36
The optimal objective is 289.964189207376
-

In [80]:
println("Region 1")
println([2,1, 5, 5, 2, 2])
println("Region 2")
println([2, 4, 0, 0, 5, 6])
println("Region 3")
println([3 , 2, 5, 0, 1, 2])
println("Region 4")
println([2, 3, 0, 0, 1, 0])
println("Region 5")
println([0, 0, 0, 5, 0, 0])
println("Region 6")
println([1, 0, 0, 0, 1, 0])

Region 1
[2, 1, 5, 5, 2, 2]
Region 2
[2, 4, 0, 0, 5, 6]
Region 3
[3, 2, 5, 0, 1, 2]
Region 4
[2, 3, 0, 0, 1, 0]
Region 5
[0, 0, 0, 5, 0, 0]
Region 6
[1, 0, 0, 0, 1, 0]
